In [48]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle
import time

In [49]:
# read metadata
path = "/media/yesindeed/WD5T/data/mimic-cxr-jpg/2.0.0/"

df = pd.read_csv(os.path.join(path, "mimic-cxr-2.0.0-metadata.csv"))
df

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,19999733,57132437,CHEST (PA AND LAT),LATERAL,3056,2544,21520708,224550.171,CHEST (PA AND LAT),lateral,Erect
377107,58766883-376a15ce-3b323a28-6af950a0-16b793bd,19999987,55368167,CHEST (PORTABLE AP),AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),antero-posterior,Erect
377108,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,19999987,58621812,CHEST (PORTABLE AP),AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),antero-posterior,Erect


In [50]:
df_label = pd.read_csv(os.path.join(path, "mimic-cxr-2.0.0-chexpert.csv"))
df_label

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227822,19999442,58708861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
227823,19999733,57132437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
227824,19999987,55368167,1.0,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
227825,19999987,58621812,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [51]:
df_patients = pd.read_csv(os.path.join(path, "patients.csv"))
df_patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,NaN
299708,19999829,F,28,2186,2008 - 2010,NaN
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,NaN


In [52]:
df_admissions = pd.read_csv(os.path.join(path, "admissions.csv"))
df_admissions = df_admissions.drop_duplicates(["subject_id"])
df_admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
5,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0
7,10000108,27250926,2163-09-27 23:17:00,2163-09-28 09:04:00,NaN,EU OBSERVATION,P38XXV,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2163-09-27 16:18:00,2163-09-28 09:04:00,0
8,10000117,22927623,2181-11-15 02:05:00,2181-11-15 14:52:00,NaN,EU OBSERVATION,P2358X,EMERGENCY ROOM,NaN,Other,ENGLISH,DIVORCED,WHITE,2181-11-14 21:51:00,2181-11-15 09:57:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431207,19999733,27674281,2152-07-08 23:29:00,2152-07-09 03:45:00,NaN,EU OBSERVATION,P63C6Q,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2152-07-08 20:15:00,2152-07-09 03:45:00,0
431208,19999784,21739106,2119-09-05 11:20:00,2119-09-08 19:00:00,NaN,ELECTIVE,P86PTJ,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,NaN,NaN,0
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1


In [53]:
df = df.merge(df_patients, how="left",
              left_on="subject_id", right_on="subject_id")
df = df.merge(df_admissions, how="left",
              left_on="subject_id", right_on="subject_id")
df = df.merge(df_label, how="left", left_on=[
              "subject_id", "study_id"], right_on=["subject_id", "study_id"])
df

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,19999733,57132437,CHEST (PA AND LAT),LATERAL,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377107,58766883-376a15ce-3b323a28-6af950a0-16b793bd,19999987,55368167,CHEST (PORTABLE AP),AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
377108,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,19999987,58621812,CHEST (PORTABLE AP),AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [54]:
df["gender"].value_counts(dropna=False)

gender
M      186739
F      170803
NaN     19568
Name: count, dtype: int64

In [55]:
df["race"].value_counts(dropna=False)

race
WHITE                                        203169
BLACK/AFRICAN AMERICAN                        47278
NaN                                           46663
OTHER                                         11721
UNKNOWN                                       10037
WHITE - OTHER EUROPEAN                         7287
HISPANIC/LATINO - PUERTO RICAN                 7200
WHITE - RUSSIAN                                5993
ASIAN - CHINESE                                4736
BLACK/CAPE VERDEAN                             4193
HISPANIC/LATINO - DOMINICAN                    3953
HISPANIC OR LATINO                             3747
ASIAN                                          3499
BLACK/CARIBBEAN ISLAND                         2403
BLACK/AFRICAN                                  1963
PORTUGUESE                                     1299
ASIAN - SOUTH EAST ASIAN                       1285
HISPANIC/LATINO - GUATEMALAN                   1019
ASIAN - ASIAN INDIAN                           1003
WHITE -

In [56]:
# remove age/sex/race == null
df = df[~df["gender"].isnull()]
# df = df[~df["anchor_age"].isnull()]
# df = df[~df["race"].isnull()]
df

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,19999733,57132437,CHEST (PA AND LAT),LATERAL,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377107,58766883-376a15ce-3b323a28-6af950a0-16b793bd,19999987,55368167,CHEST (PORTABLE AP),AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
377108,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,19999987,58621812,CHEST (PORTABLE AP),AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [57]:
df

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,19999733,57132437,CHEST (PA AND LAT),LATERAL,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377107,58766883-376a15ce-3b323a28-6af950a0-16b793bd,19999987,55368167,CHEST (PORTABLE AP),AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
377108,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,19999987,58621812,CHEST (PORTABLE AP),AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
df["No Finding"].replace(-1, 0, inplace=True)
df["No Finding"].fillna(0, inplace=True)

df

/tmp/ipykernel_752922/3878544372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["No Finding"].replace(-1, 0, inplace=True)
/tmp/ipykernel_752922/3878544372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["No Finding"].fillna(0, inplace=True)


,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,19999733,57132437,CHEST (PA AND LAT),LATERAL,3056,2544,21520708,224550.171,CHEST (PA AND LAT),...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
377107,58766883-376a15ce-3b323a28-6af950a0-16b793bd,19999987,55368167,CHEST (PORTABLE AP),AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN
377108,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,19999987,58621812,CHEST (PORTABLE AP),AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0


In [ ]:
df["No Finding"].value_counts()

No Finding
1.0    115825
Name: count, dtype: int64

In [ ]:
df.to_csv(os.path.join(path, "all.csv"), index=False)

In [60]:
len(df["dicom_id"].unique())

357542

In [61]:
def split_82(all_meta, max_test_size_per_group):
    males = all_meta[all_meta["gender"] == "M"]
    females = all_meta[all_meta["gender"] == "F"]

    males_train, males_test = train_test_split(
        np.unique(males["dicom_id"]), test_size=max_test_size_per_group, random_state=0
    )
    females_train, females_test = train_test_split(
        np.unique(females["dicom_id"]), test_size=max_test_size_per_group, random_state=0
    )

    sub_train = np.concatenate([males_train, females_train])
    sub_test = np.concatenate([males_test, females_test])

    train_meta = all_meta[all_meta["dicom_id"].isin(sub_train)]
    test_meta = all_meta[all_meta["dicom_id"].isin(sub_test)]

    return train_meta, test_meta


unique_males = df[df["gender"] == "M"]["dicom_id"].nunique()
unique_females = df[df["gender"] == "F"]["dicom_id"].nunique()
max_test_size_per_group = min(
    int(0.2 * unique_females), int(0.2 * unique_males))

# sub_train_meta, sub_test_meta = split_82(demo_data, max_test_size_per_group)


sub_train, sub_test = split_82(df, max_test_size_per_group)

sub_train.to_csv(os.path.join(path, "train.csv"), index=False)
sub_test.to_csv(os.path.join(path, "test.csv"), index=False)

In [65]:
# age

df_test = pd.read_csv(os.path.join(path, "test.csv"))

df_test = df_test[~df_test["anchor_age"].isnull()]

df_test["age_binary"] = df_test["anchor_age"].values.astype("int")
df_test["age_binary"] = np.where(
    df_test["age_binary"].between(-1, 60), 0, df_test["age_binary"])
df_test["age_binary"] = np.where(
    df_test["age_binary"] >= 60, 1, df_test["age_binary"])

class_counts = df_test["age_binary"].value_counts()
print(class_counts)
min_count = class_counts.min()
balanced_test_meta = df_test.groupby("age_binary").apply(
    lambda x: x.sample(min_count)).reset_index(drop=True)


balanced_test_meta.to_csv(os.path.join(path, "test_age.csv"), index=False)

balanced_test_meta["age_binary"].value_counts()

age_binary
1    36353
0    31967
Name: count, dtype: int64


/tmp/ipykernel_1648138/3343109671.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_test_meta = df_test.groupby("age_binary").apply(


age_binary
0    31967
1    31967
Name: count, dtype: int64

In [66]:
balanced_test_meta = balanced_test_meta.drop_duplicates(["dicom_id"])

balanced_test_meta["age_binary"].value_counts()

age_binary
0    31967
1    31967
Name: count, dtype: int64

In [67]:
# race

df_test = pd.read_csv(os.path.join(path, "test.csv"))

df_admissions = pd.read_csv(os.path.join(path, "admissions.csv"))
df_admissions = df_admissions.drop_duplicates(["subject_id"])

df_test = df_test.merge(df_admissions, how="left", left_on="subject_id", right_on="subject_id")

df_test = df_test[~df_test["race"].isnull()]
df_test["race_binary"] = df_test["race"].apply(lambda x: "White" if "WHITE" in x else "Non-white")

df_test["race_binary"].value_counts(dropna=False)

class_counts = df_test["race_binary"].value_counts()
print(class_counts)
min_count = class_counts.min()
balanced_test_meta = df_test.groupby("race_binary").apply(lambda x: x.sample(min_count)).reset_index(drop=True)


balanced_test_meta.to_csv(os.path.join(path, "test_race.csv"), index=False)
balanced_test_meta["race_binary"].value_counts()

# df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

# df_test.to_csv(os.path.join(path, "test_age.csv"), index=False)

race_binary
White        41501
Non-white    21587
Name: count, dtype: int64


/tmp/ipykernel_1648138/2450030034.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_test_meta = df_test.groupby("race_binary").apply(lambda x: x.sample(min_count)).reset_index(drop=True)


race_binary
Non-white    21587
White        21587
Name: count, dtype: int64